In [2]:
import sqlite3
import pandas as pd
import numpy as np
import ta  # For technical indicators
from sklearn.preprocessing import MinMaxScaler

# Database path
DB_PATH = "./stocks.db"
TABLE_NAME = "stock_data"
PROCESSED_TABLE = "preprocessed_stock_data"

# Connect to SQLite database
conn = sqlite3.connect(DB_PATH)

# Load the stock data
query = f"SELECT * FROM {TABLE_NAME}"
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Convert 'Date' column to datetime format
df["Date"] = pd.to_datetime(df["Date"])

# Sort data by Date & Ticker for consistency
df.sort_values(by=["Ticker", "Date"], inplace=True)

# Handle missing values using new method
df = df.ffill().bfill()

# Define function to add technical indicators
def add_technical_indicators(df):
    # Create rolling moving averages (SMA and EMA)
    df["SMA_50"] = df.groupby("Ticker")["Close"].transform(lambda x: x.rolling(window=50, min_periods=1).mean())
    df["SMA_200"] = df.groupby("Ticker")["Close"].transform(lambda x: x.rolling(window=200, min_periods=1).mean())
    df["EMA_50"] = df.groupby("Ticker")["Close"].transform(lambda x: x.ewm(span=50, adjust=False).mean())
    
    # RSI (Relative Strength Index)
    df["RSI"] = df.groupby("Ticker")["Close"].transform(lambda x: ta.momentum.RSIIndicator(x, window=14).rsi())

    # MACD (Moving Average Convergence Divergence)
    macd_indicator = ta.trend.MACD(df["Close"], window_slow=26, window_fast=12, window_sign=9)
    df["MACD"] = macd_indicator.macd()  # Extract actual MACD values

    return df

# Add technical indicators
df = add_technical_indicators(df)


# Normalize numerical columns (excluding categorical ones)
numeric_cols = ["Open", "High", "Low", "Close", "Volume", "Marketcap", "SMA_50", "SMA_200", "EMA_50", "RSI", "MACD"]
df[numeric_cols] = df[numeric_cols].astype(np.float32)
scaler_dict = {}
for ticker, group in df.groupby('Ticker'):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group[numeric_cols])
    df.loc[group.index, numeric_cols] = scaled_values
    scaler_dict[ticker] = scaler
    
# Ensure numeric_cols exist in df (some might be missing if they had NaN values)
df[numeric_cols] = scaler.fit_transform(df[numeric_cols].fillna(0))

# Connect to the database and save the processed data
conn = sqlite3.connect(DB_PATH)
df.to_sql(PROCESSED_TABLE, conn, if_exists="replace", index=False)
conn.close()

print("✅ Preprocessed data stored in SQLite successfully!")


✅ Preprocessed data stored in SQLite successfully!


In [7]:
import pandas as pd
import requests
from tqdm import tqdm

# Configuration
CSV_PATH = "nifty50_news.csv"
DEEPSEEK_API_KEY = ""
API_URL = ""

def needs_summary(existing_summary):
    """Check if summary needs regeneration"""
    if pd.isna(existing_summary):
        return True
    summary = str(existing_summary)
    return (
        "summary not available" in summary.lower() or
        "no description" in summary.lower() or
        summary.strip() in ["", "N/A", "nan"]
    )

def enhance_summary_with_deepseek(content, existing_summary):
    """Generate summary only when needed"""
    if not needs_summary(existing_summary):
        return existing_summary
    
    try:
        headers = {
            "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": "deepseek/deepseek-chat-v3-0324:free",
            "messages": [{
                "role": "user",
                "content": f"Create a 2-sentence financial news summary. Content: {str(content)[:1500]}"
            }]
        }
        
        response = requests.post(API_URL, headers=headers, json=data, timeout=15)
        
        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content']
            
        return existing_summary
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return existing_summary

def preprocess_data():
    # Load and clean data
    df = pd.read_csv(CSV_PATH)
    df = df.dropna(subset=['Title', 'URL'])
    df = df.drop_duplicates(subset=['URL'])
    
    # Process summaries
    tqdm.pandas(desc="Processing Articles")
    mask = df['Description'].apply(needs_summary)
    print(f"Generating {sum(mask)}/{len(df)} summaries needed")
    
    df.loc[mask, 'summary'] = df[mask].progress_apply(
        lambda row: enhance_summary_with_deepseek(row['Content'], row['Description']), 
        axis=1
    )
    df.loc[~mask, 'summary'] = df.loc[~mask, 'Description']
    
    # Save processed data to new CSV
    df.to_csv("processed_news.csv", index=False)
    print("✅ Preprocessing complete! Saved to processed_news.csv")

# Run this cell to preprocess data
preprocess_data()

Generating 11/871 summaries needed


Processing Articles: 100%|██████████| 11/11 [00:40<00:00,  3.72s/it]

✅ Preprocessing complete! Saved to processed_news.csv


In [26]:
import sqlite3
import pandas as pd
from tqdm import tqdm

def insert_to_db():
    DB_PATH = "finance_news.db"
    
    try:
        # Load processed data
        df = pd.read_csv("processed_news.csv")
        
        # Connect to database
        conn = sqlite3.connect(DB_PATH)
        
        # Get existing URLs
        existing_urls = pd.read_sql("SELECT url FROM nifty_fifty_news", conn)['url'].tolist()
        print(f"Found {len(existing_urls)} existing articles in DB")
        
        # Filter new articles
        new_articles = df[~df['URL'].isin(existing_urls)]
        print(f"Inserting {len(new_articles)} new articles")
        
        if not new_articles.empty:
            # Prepare data for DB
            new_articles = new_articles.rename(columns={
                'Stock': 'stock',
                'Sector': 'sector',
                'Title': 'headline',
                'Source': 'source',
                'Date': 'published_date',
                'URL': 'url'
            })
            
            # Insert in chunks with progress
            chunks = [new_articles[i:i+100] for i in range(0, len(new_articles), 100)]
            with tqdm(total=len(new_articles), desc="Inserting to DB") as pbar:
                for chunk in chunks:
                    chunk.to_sql('nifty_fifty_news', conn, 
                                if_exists='append', 
                                index=False,
                                method='multi')
                    pbar.update(len(chunk))
            print("✅ Database update complete!")
        else:
            print("💡 No new articles to insert")
            
    except Exception as e:
        print(f"🚨 Error: {str(e)}")
    finally:
        if 'conn' in locals():
            conn.close()

# Run this cell separately to update database
insert_to_db()

Found 871 existing articles in DB
Inserting 0 new articles
💡 No new articles to insert


In [32]:
import sqlite3
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from tqdm import tqdm

# Load FinBERT model
MODEL_NAME = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
sentiment_analyzer = pipeline("text-classification", model=model, tokenizer=tokenizer)

def analyze_sentiment(text):
    """Get sentiment score using FinBERT"""
    try:
        # Truncate text to model's max length (512 tokens)
        truncated_text = text[:4000]  # Conservative truncation
        result = sentiment_analyzer(truncated_text)[0]
        # print("score=", result['score'])
        return {
            'sentiment': result['label'],
            'score': result['score']
        }
    except Exception as e:
        print(f"Error analyzing sentiment: {str(e)}")
        return {'sentiment': 'neutral', 'score': 0.0}

def update_sentiment_scores():
    DB_PATH = "finance_news.db"
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Get all articles needing sentiment analysis
        df = pd.read_sql("""
            SELECT id, headline, summary 
            FROM nifty_fifty_news 
        """, conn)
        
        print(f"Found {len(df)} articles needing sentiment analysis")
        
        # Process in batches
        batch_size = 100
        updated_count = 0
        
        with tqdm(total=len(df), desc="Analyzing Sentiment") as pbar:
            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]
                updates = []
                
                for _, row in batch.iterrows():
                    # Combine headline and summary for better context
                    text = f"{row['headline']}. {row['summary']}"
                    sentiment = analyze_sentiment(text)
                    
                    updates.append((
                        sentiment['sentiment'],
                        sentiment['score'],
                        row['id']
                    ))
                    pbar.update(1)
                
                # Batch update
                cursor.executemany("""
                    UPDATE nifty_fifty_news
                    SET sentiment = ?, sentiment_score = ?
                    WHERE id = ?
                """, updates)
                conn.commit()
                updated_count += len(updates)
        
        print(f"✅ Updated {updated_count} records with sentiment scores")
        
    except Exception as e:
        print(f"🚨 Error: {str(e)}")
    finally:
        if conn:
            conn.close()

# Run the sentiment analysis
update_sentiment_scores()

Device set to use cpu


Found 871 articles needing sentiment analysis


Analyzing Sentiment: 100%|██████████| 871/871 [02:24<00:00,  6.02it/s]

✅ Updated 871 records with sentiment scores


In [16]:
!pip install transformers torch sentencepiece

  Using cached transformers-4.50.2-py3-none-any.whl.metadata (39 kB)
  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached transformers-4.50.2-py3-none-any.whl (10.2 MB)
Using cached torch-2.6.0-cp311-cp311-win_amd64.whl (204.2 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--